In [2]:
import networkx as nx
from node2vec import Node2Vec
from igraph import Graph
import igraph as ig
import numpy as np
import json
import pandas as pd
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GraphSAGE
from torch_geometric.data import DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import GAE
from torch_geometric.nn import VGAE
import neptune
import wandb
import os

In [3]:
node_list_path = r'C:\Users\George\Desktop\ISEF-2023\Datas\Node list\back up\current_protein_Signal+meta+targets.json'
with open(node_list_path, 'r') as file:
    node_list = json.load(file)
graph = r'C:\Users\George\Desktop\ISEF-2023\Network construction\PPI_homo_graph_features_loaded.graphml'
# Create an igraph object
PPI_graph = ig.Graph.Load(graph, format='graphml')

feature_keys = [
    "Indegree", "Outdegree", "Closeness", "Betweenness", "Pagerank", "Cluster_coefficients",
    "Nearest_Neighbor_Degree", "Similarity", "Subunit", "Transmembrane",
    "Catalytic_activity", "Interaction", "Tissue_Specificity", "Disease",
    "Sequence_conflict", "Modified_residue", "Function", "Binding_site",
    "Natural_variant", "Alternative_products", "Subcellular_location",
    "Active_site", "Disulfide_bond", "Mutagenesis", "PTM", "STP_involvement"
]

features = torch.tensor([
    PPI_graph.vs[key] for key in feature_keys
], dtype=torch.float).t()

edge_indices = torch.tensor(PPI_graph.get_edgelist(), dtype=torch.long).t()

# Assuming you have a label attribute in your graph
labels = torch.tensor(PPI_graph.vs["label"], dtype=torch.float)

# Create a PyTorch Geometric Data object
data = Data(x=features, edge_index=edge_indices, y=labels)

print(data)

Data(x=[7392, 26], edge_index=[2, 49502], y=[7392])


C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\igraph\io\files.py:295: RuntimeWarning: Could not add vertex ids, there is already an 'id' vertex attribute. at src/io/graphml.c:492
  return reader(f, *args, **kwds)


In [4]:

data = train_test_split_edges(data)
print(data)

C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Data(x=[7392, 26], y=[7392], val_pos_edge_index=[2, 1237], test_pos_edge_index=[2, 2474], train_pos_edge_index=[2, 42062], train_neg_adj_mask=[7392, 7392], val_neg_edge_index=[2, 1237], test_neg_edge_index=[2, 2474])


In [5]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, layer_channels):
        super(VariationalGCNEncoder, self).__init__()

        self.convs = torch.nn.ModuleList()

        # Add the input layer
        self.convs.append(GCNConv(in_channels, layer_channels[0], cached=True))
        # Add hidden layers
        for i in range(1, len(layer_channels)-1):
            self.convs.append(GCNConv(layer_channels[i-1], layer_channels[i], cached=True))
        
        self.conv_mu = GCNConv(layer_channels[-2], layer_channels[-1], cached=True)
        self.conv_logstd = GCNConv(layer_channels[-2], layer_channels[-1], cached=True)

    def forward(self, x, edge_index):
        for conv in self.convs[:-1]:
            x = conv(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)
    

In [16]:
run = neptune.init_run(
    project="georgewang2008/VGAE",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJkNjU3MjQyMy0yZWUzLTQzNGYtOTI3MS0zNDNlYjJhOWQ3ZTgifQ==",
)

num_features = 26
epochs = 240
layer_channels = [32, 64, 64, 128]  # List of channel sizes for each layer
lr = 0.01

param = {
    'num_features':num_features,
    'epochs':epochs,
    'layer_channels':str(layer_channels),  # List of channel sizes for each layer
    'lr':lr
}
run["parameters"] = param

# model

model = VGAE(VariationalGCNEncoder(in_channels=num_features, layer_channels=layer_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr)

https://app.neptune.ai/georgewang2008/VGAE/e/ABC-12


In [17]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [18]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

    run["train/loss"].append(loss)
    run["train/auc"].append(auc)
    run["train/ap"].append(ap)
    run["train/epoch"].append(epoch)

    #if auc > 91 and ap > 91.1:
        #break

Epoch: 001, AUC: 0.5200, AP: 0.5102
Epoch: 002, AUC: 0.5394, AP: 0.5205
Epoch: 003, AUC: 0.5346, AP: 0.5179
Epoch: 004, AUC: 0.5315, AP: 0.5163
Epoch: 005, AUC: 0.5315, AP: 0.5163
Epoch: 006, AUC: 0.5315, AP: 0.5163
Epoch: 007, AUC: 0.5331, AP: 0.5171
Epoch: 008, AUC: 0.5352, AP: 0.5182
Epoch: 009, AUC: 0.5356, AP: 0.5184
Epoch: 010, AUC: 0.5360, AP: 0.5187
Epoch: 011, AUC: 0.5368, AP: 0.5191
Epoch: 012, AUC: 0.5380, AP: 0.5197
Epoch: 013, AUC: 0.5384, AP: 0.5200
Epoch: 014, AUC: 0.5396, AP: 0.5206
Epoch: 015, AUC: 0.5420, AP: 0.5219
Epoch: 016, AUC: 0.5440, AP: 0.5230
Epoch: 017, AUC: 0.5440, AP: 0.5230
Epoch: 018, AUC: 0.5416, AP: 0.5217
Epoch: 019, AUC: 0.5410, AP: 0.5214
Epoch: 020, AUC: 0.5412, AP: 0.5215
Epoch: 021, AUC: 0.5507, AP: 0.5267
Epoch: 022, AUC: 0.5562, AP: 0.5299
Epoch: 023, AUC: 0.5523, AP: 0.5278
Epoch: 024, AUC: 0.5532, AP: 0.5282
Epoch: 025, AUC: 0.5451, AP: 0.5236
Epoch: 026, AUC: 0.5402, AP: 0.5210
Epoch: 027, AUC: 0.5400, AP: 0.5208
Epoch: 028, AUC: 0.5400, AP:

Epoch: 229, AUC: 0.5476, AP: 0.5251
Epoch: 230, AUC: 0.6100, AP: 0.5632
Epoch: 231, AUC: 0.5760, AP: 0.5414
Epoch: 232, AUC: 0.7073, AP: 0.6345
Epoch: 233, AUC: 0.5563, AP: 0.5301
Epoch: 234, AUC: 0.5869, AP: 0.5484
Epoch: 235, AUC: 0.6986, AP: 0.6276
Epoch: 236, AUC: 0.6096, AP: 0.5629
Epoch: 237, AUC: 0.6767, AP: 0.6109
Epoch: 238, AUC: 0.6932, AP: 0.6228
Epoch: 239, AUC: 0.6102, AP: 0.5629
Epoch: 240, AUC: 0.5461, AP: 0.5243


In [9]:
Z = model.encode(x, train_pos_edge_index)
Z

tensor([[-11.9406,  12.3173,  13.3241, -13.2107,  34.4085,  20.9689],
        [ -0.7754,  -1.6070,   0.5959,   6.5201,  -3.2338,  -0.7822],
        [ -0.4110,   2.4585,  -1.1412,  -4.9950,   4.5098,   7.6872],
        ...,
        [ -2.9609,  -3.6318,   2.5026,  18.0569,  -8.0509,  -1.6856],
        [-22.2418,  22.3279,  26.5059, -21.1681,  63.0139,  33.4849],
        [-14.5902,  14.5423,  17.1571, -14.2777,  41.6708,  23.1909]],
       grad_fn=<AddBackward0>)

In [ ]:
import csv
node_names = PPI_graph.vs["name"]
csv_data = [["NodeName"] + [f"Feature_{i}" for i in range(Z.shape[1])]]

for name, embedding in zip(node_names, Z):
    row = [name] + embedding.tolist()
    csv_data.append(row)

print(csv_data)

csv_file_path = r"C:\Users\George\Desktop\ISEF-2023\Model\test\VGAE_PPI\embed_38.csv"

# Write the CSV file
with open(csv_file_path, "w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(csv_data)

print(f"CSV file saved at: {csv_file_path}")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



CSV file saved at: C:\Users\George\Desktop\ISEF-2023\Model\test\GAE_PPI\embed_38.csv
Unexpected error occurred in Neptune background thread: Killing Neptune ping thread. Your run's status will not be updated and the run will be shown as inactive.


Exception in thread NeptunePing:
Traceback (most recent call last):
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\neptune\internal\backends\swagger_client_wrapper.py", line 111, in __call__
    return FinishedApiResponseFuture(future.response())  # wait synchronously
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 200, in response
    swagger_result = self._get_swagger_result(incoming_response)
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 124, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 300, in _get_swagger_result
    unmarshal_response(
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 353, in unmarshal_response
    raise_on_expected(incoming_r

Unexpected error occurred in Neptune background thread: Killing Neptune ping thread. Your run's status will not be updated and the run will be shown as inactive.


Exception in thread NeptunePing:
Traceback (most recent call last):
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\neptune\internal\backends\swagger_client_wrapper.py", line 111, in __call__
    return FinishedApiResponseFuture(future.response())  # wait synchronously
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 200, in response
    swagger_result = self._get_swagger_result(incoming_response)
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 124, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 300, in _get_swagger_result
    unmarshal_response(
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 353, in unmarshal_response
    raise_on_expected(incoming_r

Unexpected error occurred in Neptune background thread: Killing Neptune ping thread. Your run's status will not be updated and the run will be shown as inactive.


Exception in thread NeptunePing:
Traceback (most recent call last):
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\neptune\internal\backends\swagger_client_wrapper.py", line 111, in __call__
    return FinishedApiResponseFuture(future.response())  # wait synchronously
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 200, in response
    swagger_result = self._get_swagger_result(incoming_response)
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 124, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 300, in _get_swagger_result
    unmarshal_response(
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 353, in unmarshal_response
    raise_on_expected(incoming_r

Unexpected error occurred in Neptune background thread: Killing Neptune ping thread. Your run's status will not be updated and the run will be shown as inactive.


Exception in thread NeptunePing:
Traceback (most recent call last):
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\neptune\internal\backends\swagger_client_wrapper.py", line 111, in __call__
    return FinishedApiResponseFuture(future.response())  # wait synchronously
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 200, in response
    swagger_result = self._get_swagger_result(incoming_response)
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 124, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 300, in _get_swagger_result
    unmarshal_response(
  File "C:\Users\George\AppData\Local\Programs\Python\Python310\lib\site-packages\bravado\http_future.py", line 353, in unmarshal_response
    raise_on_expected(incoming_r